In [1]:
import json 
import os
import logging 

from datetime import timedelta
from pathlib import Path 
from pprint import PrettyPrinter

from subgrounds import Subgrounds
from web3 import Web3
from pycoingecko import CoinGeckoAPI

from prefect import task, flow 
from prefect.client import get_client
from prefect.tasks import task_input_hash

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss

from flywheel_util.constants import (
    colors_24,
    colors_28, 
    addresses, 
    url_infura, 
    url_snapshot, 
    url_subgraphs, 
    snapshot_api_max_records_per_request, 
    snapshot_api_max_skip,
)
from flywheel_util.tasks.general import df_to_sql
from flywheel_util.utils.util import (
    ddf, 
    first_row, 
    query_attrs,
    recursive_index_merge, 
)

from sqlalchemy import create_engine
from sqlalchemy import text
engine = create_engine("sqlite+pysqlite:///votium_bribes.db", echo=False, future=True)

# logging.basicConfig(level=logging.INFO)

import logging
logging.basicConfig(level=logging.DEBUG)

pp = PrettyPrinter().pprint

alt.data_transformers.disable_max_rows()

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/eth_utils/toolz.py:2: DeprecationWarning: The toolz.compatibility module is no longer needed in Python 3 and has been deprecated. Please import these utilities directly from the standard library. This module will be removed in a future release.
  from cytoolz import (
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/web3/_utils/normalizers.py:231: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(eth_abi.__version__) < LooseVersion("2"):


DataTransformerRegistry.enable('default')

In [2]:
w3 = Web3(Web3.HTTPProvider(url_infura))
cg = CoinGeckoAPI()

In [3]:
sg = Subgrounds()
sg_curve_vol = sg.load_subgraph(url_subgraphs.convex.curve_vol_mainnet)

In [4]:
url_subgraphs.convex.curve_vol_mainnet

'https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet'

In [19]:
@task(persist_result=True, cache_key_fn=task_input_hash, cache_expiration=timedelta(days=1))
def get_tvl_against_base_metapool_asset(metapool_lp_token): 
    # Pull data from curve subgraph 
    query_pools = sg_curve_vol.Query.pools(
        first=1000, where={"coins_contains": [metapool_lp_token]}
    )
    query_snaps = query_pools.dailyPoolSnapshots(first=1, orderBy="timestamp", orderDirection="desc") 
    dfs = sg.query_df([
        query_pools.name, 
        query_pools.coins, 
        query_pools.coinNames, 
        query_snaps.reserves, 
        query_snaps.reservesUSD, 
    ])
    df = recursive_index_merge(dfs)
    df = df.loc[df.pools_dailyPoolSnapshots_reservesUSD > 0]
    df = df.loc[df.pools_coinNames.str.lower() != '3crv']
    df = df.groupby('pools_name')['pools_dailyPoolSnapshots_reservesUSD'].sum().reset_index()
    df = df.rename(columns={"pools_dailyPoolSnapshots_reservesUSD": "reserves_usd"})
    return df

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/tasks.py:206: UserWarning: A task named 'get_tvl_against_base_metapool_asset' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_91036/4192669198.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [20]:
@flow 
def flow_fraxbp_vs_3pool(): 
    df_fraxbp = get_tvl_against_base_metapool_asset.submit(addresses.token.crvfrax)
    df_3pool = get_tvl_against_base_metapool_asset.submit(addresses.token['3crv'])
    return df_fraxbp.result(), df_3pool.result()

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/prefect/flows.py:205: UserWarning: A flow named 'flow-fraxbp-vs-3pool' and defined at '/var/folders/5c/hf9_blgj3jbbxxvkw_s9snsw0000gn/T/ipykernel_91036/2189869363.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [21]:
df_fraxbp, df_3pool = flow_fraxbp_vs_3pool()

14:20:53.537 | INFO    | prefect.engine - Created flow run 'hysterical-oxpecker' for flow 'flow-fraxbp-vs-3pool'

14:20:53.604 | INFO    | Flow run 'hysterical-oxpecker' - Created task run 'get_tvl_against_base_metapool_asset-800861d2-0' for task 'get_tvl_against_base_metapool_asset'

14:20:53.605 | INFO    | Flow run 'hysterical-oxpecker' - Submitted task run 'get_tvl_against_base_metapool_asset-800861d2-0' for execution.

14:20:53.614 | INFO    | Flow run 'hysterical-oxpecker' - Created task run 'get_tvl_against_base_metapool_asset-800861d2-1' for task 'get_tvl_against_base_metapool_asset'

14:20:53.614 | INFO    | Flow run 'hysterical-oxpecker' - Submitted task run 'get_tvl_against_base_metapool_asset-800861d2-1' for execution.

14:21:04.699 | INFO    | Task run 'get_tvl_against_base_metapool_asset-800861d2-0' - Finished in state Completed()

14:21:47.346 | INFO    | Task run 'get_tvl_against_base_metapool_asset-800861d2-1' - Finished in state Completed()

14:21:47.356 | INFO    | Flow run 'hysterical-oxpecker' - Finished in state Completed()

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x16ba321a0>


In [22]:
def chart_tvl_breakdown(df, asset_name): 
    print(f"Total tvl ($) paired against {asset_name} in Metapools: ${df.reserves_usd.sum():,}")
    return (
        alt.Chart(df)
        .mark_arc()
        .encode(
            theta="reserves_usd", 
            color="pools_name", 
            tooltip=["pools_name", alt.Tooltip("reserves_usd", format=",d")]
        )
        .properties(title=f"Breakdown of TVL paired against {asset_name} in Metapools")
    ) 


In [23]:
chart_tvl_breakdown(df_fraxbp, "crvFRAX")

Total tvl ($) paired against crvFRAX in Metapools: $123,769,995.96578994


/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [24]:
chart_tvl_breakdown(df_3pool, "3Crv")

Total tvl ($) paired against 3Crv in Metapools: $493,890,025.5589819


alt.Chart(...)